In [1]:
import pandas as pd 
import json
import pymol2
import requests

Files `entries.json`, `RepeatsDB-table.tsv` and `RepeatsDB-table.mfas`  can be downloaded at the link https://repeatsdb.bio.unipd.it/search

In [2]:
with open('entries.json') as json_file:
    data = json.load(json_file)
labels = pd.read_csv("labels.csv", header = None, names = ["id", "name"])
labels = pd.DataFrame(labels)

In [72]:
uniprot_df = pd.read_csv("RepeatsDB-table.tsv", header = None, sep = "\t")
dic_pdb = {}
uniprot_df = pd.DataFrame(uniprot_df)
uniprot_df.head()
for idx, el in uniprot_df[1:].iterrows():
    list_pdb = el[4].split(",")
    for pdb in list_pdb :
        dic_pdb[pdb] = el[2]
print(dic_pdb)

{'1pbyA': 'Q8VUT0', '3oyoA': 'F2Z290', '3nvsA': 'Q9KRB0', '3ti2A': 'Q9KRB0', '3ti2B': 'Q9KRB0', '3ti2C': 'Q9KRB0', '3ti2D': 'Q9KRB0', '3v64C': 'Q9QYP1', '3v64D': 'Q9QYP1', '3v65B': 'Q9QYP1', '3v65D': 'Q9QYP1', '3eweA': 'P53011', '3eweC': 'P53011', '3f3gA': 'P53011', '3f3gB': 'P53011', '3f3gE': 'P53011', '3f3gF': 'P53011', '3f3pA': 'P53011', '3f3pB': 'P53011', '3f3pE': 'P53011', '3f3pF': 'P53011', '3f3pI': 'P53011', '3f3pJ': 'P53011', '4xmmC': 'P53011', '5ewpA': 'W7FPA1', '5ewpB': 'W7FPA1', '1znnA': 'Q5L3Y2', '1znnB': 'Q5L3Y2', '1znnC': 'Q5L3Y2', '1znnD': 'Q5L3Y2', '1znnE': 'Q5L3Y2', '1znnF': 'Q5L3Y2', '4wxzA': 'Q5L3Y2', '4wxzB': 'Q5L3Y2', '4wxzC': 'Q5L3Y2', '4wxzD': 'Q5L3Y2', '4wxzE': 'Q5L3Y2', '4wxzF': 'Q5L3Y2', '4wy0A': 'Q5L3Y2', '4wy0B': 'Q5L3Y2', '4wy0C': 'Q5L3Y2', '4wy0D': 'Q5L3Y2', '4wy0E': 'Q5L3Y2', '4wy0F': 'Q5L3Y2', '4wy0G': 'Q5L3Y2', '4wy0H': 'Q5L3Y2', '4wy0I': 'Q5L3Y2', '4wy0J': 'Q5L3Y2', '4wy0K': 'Q5L3Y2', '4wy0L': 'Q5L3Y2', '1ujpA': 'P16608', '1wxjA': 'P16608', '1knlA': 'P

In [87]:
dic_uniprots = {}
with open ("RepeatsDB-table.mfas") as f :
    for line in f:
        if line.startswith(">") :
            line = line.strip()
            u = line.split(" ")
            dic_uniprots[u[1]] = next(f).strip()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
print(labels)

          id                                    name
0          2                          Fibrous repeat
1        2.1                   Collagen triple-helix
2      2.1.0                                Collagen
3    2.1.0.0                                Collagen
4        2.2               Alpha helical coiled coil
..       ...                                     ...
128    5.4.0          Repeat of beta sandwich domain
129  5.4.0.0        Unclassified beta-sandwich beads
130      5.5               Alpha/beta sandwich beads
131    5.5.0     Repeat of alpha/beta sandwich beads
132  5.5.0.0  Unclassified alpha/beta-sandwich beads

[133 rows x 2 columns]


In [95]:
dic_pdbchain = {}
for element in data :
    if element["pdb_id"] + element["pdb_chain"] in dic_pdbchain :
        if element["type"] == "unit" :
            dic_pdbchain[element["pdb_id"] + element["pdb_chain"]][9].append([element["start"], element["end"]])
            dic_pdbchain[element["pdb_id"] + element["pdb_chain"]][9].sort(key=lambda x: (int(x[0])))
            dic_pdbchain[element["pdb_id"] + element["pdb_chain"]][7] = dic_pdbchain[element["pdb_id"] + element["pdb_chain"]][9][0][0]
            dic_pdbchain[element["pdb_id"] + element["pdb_chain"]][8] = dic_pdbchain[element["pdb_id"] + element["pdb_chain"]][9][-1][1]
        if element["type"] == "insertion" :
            dic_pdbchain[element["pdb_id"] + element["pdb_chain"]][10].append([element["start"], element["end"]])
            dic_pdbchain[element["pdb_id"] + element["pdb_chain"]][10].sort(key=lambda x: (int(x[0])))
    else :
        if element["type"] == "unit" :
            if (element["pdb_id"] + element["pdb_chain"] in dic_pdb.keys()) :
                sequence = dic_uniprots[dic_pdb[element["pdb_id"] + element["pdb_chain"]]]
            else :
                sequence = None
            if len(labels[labels["id"] == element["class"]]["name"]) > 0 :
                reg_class = labels[labels["id"] == element["class"]]["name"].values[0]
            else :
                reg_class = None
            if len(labels[labels["id"] == element["class_topology"]]["name"]) > 0 :
                reg_top = labels[labels["id"] == element["class_topology"]]["name"].values[0]
            else :
                reg_top = None
            if len(labels[labels["id"] == element["class_topology_fold"]]["name"]) > 0 :
                reg_fold = labels[labels["id"] == element["class_topology_fold"]]["name"].values[0]
            else :
                reg_fold = None
            if len(labels[labels["id"] == element["class_topology_fold_clan"]]["name"]) > 0 :
                reg_clan = labels[labels["id"] == element["class_topology_fold_clan"]]["name"].values[0]
            else :
                reg_clan = None
            dic_pdbchain.setdefault(element["pdb_id"] + element["pdb_chain"], [element["pdb_id"], element["pdb_chain"], sequence, reg_class, reg_top,
                               reg_fold, reg_clan, element["start"], element["end"], [[element["start"], 
                               element["end"]]], []])

In [96]:
print(dic_pdbchain["6kivR"])
dic_pdbchain_df = pd.DataFrame(dic_pdbchain.values(), columns=['PDB', "chain", "sp_primary", "class", "topology", "fold", "clan", "reg_start", "reg_end", "units", "insertions"])

['6kiv', 'R', 'MATEEKKPETEAARAQPTPSSSATQSKPTPVKPNYALKFTLAGHTKAVSSVKFSPNGEWLASSSADKLIKIWGAYDGKFEKTISGHKLGISDVAWSSDSNLLVSASDDKTLKIWDVSSGKCLKTLKGHSNYVFCCNFNPQSNLIVSGSFDESVRIWDVKTGKCLKTLPAHSDPVSAVHFNRDGSLIVSSSYDGLCRIWDTASGQCLKTLIDDDNPPVSFVKFSPNGKYILAATLDNTLKLWDYSKGKCLKTYTGHKNEKYCIFANFSVTGGKWIVSGSEDNLVYIWNLQTKEIVQKLQGHTDVVISTACHPTENIIASAALENDKTIKLWKSDC', 'Closed repeat', 'Propeller', 'Seven blade propeller', 'WD40 repeat', '47', '302', [['47', '88'], ['89', '128'], ['129', '170'], ['171', '215'], ['216', '256'], ['257', '302']], []]


In [97]:
dic_pdbchain_df = dic_pdbchain_df.rename(columns={'sequence': 'sp_primary'})
dic_pdbchain_df.head()

,PDB,chain,sp_primary,class,topology,fold,clan,reg_start,reg_end,units,insertions
0,4u3j,C,MSGEEEVDYTTLPLEERLTYKLWKARLEAYKELNQLFRNSVGDISR...,Elongated repeat,Alpha-solenoid,Unclassified alpha solenoid,Unclassified alpha solenoid,326,536,"[[326, 363], [364, 408], [409, 448], [449, 491...",[]
1,3u3w,A,MQAEKLGSEIKKIRVLRGLTQKQLSENICHQSEVSRIESGAVYPSM...,Elongated repeat,Alpha-solenoid,Unclassified alpha solenoid,Unclassified alpha solenoid,3,271,"[[3, 43], [44, 57], [58, 71], [72, 108], [109,...",[]
2,6u3w,B,MDYFNIKQNYYTGNFVQCLQEIEKFSKVTDNTLLFYKAKTLLALGQ...,Elongated repeat,Alpha-solenoid,Unclassified alpha solenoid,Unclassified alpha solenoid,14,290,"[[14, 46], [68, 98], [99, 134], [135, 163], [1...","[[47, 67], [164, 179]]"
3,3u3w,B,MQAEKLGSEIKKIRVLRGLTQKQLSENICHQSEVSRIESGAVYPSM...,Elongated repeat,Alpha-solenoid,Unclassified alpha solenoid,Unclassified alpha solenoid,72,270,"[[72, 108], [109, 150], [151, 190], [191, 231]...",[]
4,3u3h,A,MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVRRLAEL...,Closed repeat,Alpha/beta prism,Alpha-beta prism,Alpha-beta prism,9,382,"[[9, 47], [48, 86], [87, 138], [139, 181], [18...","[[182, 194], [216, 240], [325, 370]]"


In [98]:
dic_pdbchain_df.to_csv("all_reviewed_dataset.csv")